# Import

In [ ]:
# Gym
import os
import random
from typing import Any

# helpers
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Box, Dict, Discrete, MultiBinary, MultiDiscrete, Tuple

# Stable-baseline
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

# Types of Spaces

In [ ]:
Discrete(3).sample()

In [ ]:
Box(0, 1, shape=(3,3)).sample()

In [ ]:
Tuple((Discrete(3), Box(0, 1, shape=(3,3)))).sample()

In [ ]:
Dict({"height" : Discrete(3), "place" : Box(0,1, shape=(1,2)), "tuple" : Tuple((Discrete(3), Box(0, 1, shape=(3,3))))}).sample()

In [ ]:
MultiBinary((4,2)).sample()

In [ ]:
MultiDiscrete([2,3,5]).sample()

# Building Environment

In [ ]:
Box(low=np.array([0]), high=np.array([100])).sample()

In [ ]:
Box(low=0, high=100).sample()

In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100)
        self.state = float(38 + random.randint(-3,3))
        self.shower_length = 60

    def step(self, action):
        action_mapping = {0: -1, 1: 0, 2: 1}
        temp_change = action_mapping[action]
        self.state += temp_change

        self.shower_length -= 1

        MIN_TEMP = 37
        MAX_TEMP = 39
        reward = 1 if self.state >= MIN_TEMP and self.state <= MAX_TEMP else -1

        reward = -abs(self.state - 38)

        if self.shower_length <= 0:
            terminated = True
            truncated = True
        else:
            terminated = False
            truncated = False


        info = {}

        return np.array([self.state], dtype=np.float32), reward, terminated, truncated, info

    def render(self):
        pass

    def reset(
        self,
        *,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ):
        self.state = float(38 + random.randint(-3,3))
        self.shower_length = 60
        info = {}

        return self.state, info

In [ ]:
env = ShowerEnv()

In [ ]:
env.observation_space.sample()

In [ ]:
env.action_space.sample()

# Test Environment

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    env.reset()
    terminated = False
    truncated = False
    score = 0

    while not terminated and not truncated:
        action = env.action_space.sample()
        obs, reward, terminated, truncated , info = env.step(action)
        score += reward
    print(f"Episode: {episode}  Score {score}")
env.close()

# Train

In [ ]:
log_path = os.path.join("training","logs")
model = DQN("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(100000, reset_num_timesteps=False, progress_bar=True)

# Save

In [ ]:
shower_path = os.path.join("training","model","Shower_Model_PPO")

In [ ]:
model.save(shower_path)

In [ ]:
del model

In [ ]:
model = DQN.load(shower_path, env)

In [ ]:
evaluate_policy(model, env, 100)